In [1]:
import sys
#from google.colab import drive
#drive.mount('/content/drive')
#sys.path.append('/content/drive/MyDrive/TCM-QA')
#sys.path.append('/content/drive/MyDrive/TCM-QA/latticeNER')
#sys.path.append('/content/drive/MyDrive/TCM-QA/BertIR')
sys.path.append('E:/毕业论文相关/毕业论文/代码/TCM-QA')
sys.path.append('E:/毕业论文相关/毕业论文/代码/TCM-QA/latticeNER')
sys.path.append('E:/毕业论文相关/毕业论文/代码/TCM-QA/BertIR')

sys.argv=['']
del sys

In [2]:
import jieba

In [3]:
import LatticeNER_run
import torch
import pickle

import BertIR_run

import SiameseLSTMEL_run

D:\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import os
from py2neo import Graph,Node
import pandas as pd

In [5]:
class Graph2Answer:
    def __init__(self):
        """
        self.g = Graph(
            host="7474",  # neo4j 搭载服务器的ip地址，ifconfig可获取到
            http_port=7474,  # neo4j 服务器监听的端口号
            user="neo4j",  # 数据库user name，如果没有更改过，应该是neo4j
            password="password")
        """
        self.g = Graph("http://localhost:7474", auth=("neo4j","password"))
    def Q_sym_med(self,sym_detail):
        """
        已知症状问药方名
        """
        query = "match (n:Symptom)-[r:治疗]-(m:Name) where n.name = '%s' return m.name" % (sym_detail)
        ans = self.g.run(query)
        return ans.to_data_frame()
    
    def Q_med_con(self,name_detail):
        """
        已知药方名问中药和剂量
        """
        query = "match (n:Name)-[r:包含]-(m:Drug) where n.name = '%s' return m.name,r.分量" % (name_detail)
        ans = self.g.run(query)
        
        return ans.to_data_frame()
    
    def Q_med_use(self,name_detail):
        """
        已知药方名问用法，就是那一长句
        """
        query = "match (n:Name)-[r:使用方法]-(m:Use) where n.name = '%s' return r.起始索引,m.name" % (name_detail)
        ans = self.g.run(query)
        df = ans.to_data_frame()
        df = df.apply(pd.to_numeric, errors='ignore')
        df = df.sort_values(by = "r.起始索引",ascending=True)
        return df
    
    def Q_med_sym(self,name_detail):
        """
        已知药方名问治疗什么症状
        """
        query = "match (n:Name)-[r:治疗]-(m:Symptom) where n.name = '%s' return m.name" % (name_detail)
        ans = self.g.run(query)
        return ans.to_data_frame()

In [6]:
class TCM_QA:
    def __init__(self):
        #路径initial
        print('正在进行系统初始化···')
        self.ner_model_path = "E:/毕业论文相关/毕业论文/代码/TCM-QA/NER data/417final.model"
        self.ner_data_path = "E:/毕业论文相关/毕业论文/代码/TCM-QA/NER data/model-train-data_final417.dset"
        self.ques_temp_path = "E:/毕业论文相关/毕业论文/代码/TCM-QA/NER data/ques.txt"
        self.qa_sym_path = "E:/毕业论文相关/毕业论文/代码/TCM-QA/KG data/SYM_CORPUS.txt"
        self.qa_med_path = "E:/毕业论文相关/毕业论文/代码/TCM-QA/KG data/NAM_CORPUS.txt"
        #######################
        self.ner_model=torch.load(self.ner_model_path)
        with open(self.ner_data_path, 'rb') as fp:
            self.ner_data = pickle.load(fp)
        self.gpu = torch.cuda.is_available()

        self.el_model = SiameseLSTMEL_run.SiameseNetwork()

        f=open(self.qa_sym_path,'r',encoding='utf-8')
        temp = f.readlines()
        self.qa_sym = []
        for i in temp:
            self.qa_sym.append(i.replace('\n',''))

        f=open(self.qa_med_path,'r',encoding='utf-8')
        temp = f.readlines()
        self.qa_med = []
        for i in temp:
            self.qa_med.append(i.replace('\n',''))
        
        self.kg = Graph2Answer()

    def ner(self):
        f=open(self.ques_temp_path,'w',encoding='utf-8')
        for i in self.ques:
            f.write(i+'	'+'O'+'\n')
        f.write('\n')
        f.close()
        self.ner_data.generate_instance_with_gaz(self.ques_temp_path,'test')
        self.ner_pred_results = LatticeNER_run.load_model_decode_with_model_predict(self.ner_model, self.ner_data, 'test', self.gpu, True)
        #[['B-SYM','E-SYM','O','O','O','O','O','O','B-MED','I-MED','E-MED','O','O','O','O']]

    def ir(self):
        self.ir_pred_results = BertIR_run.predict(self.intent_label_str)

    def el(self,s1,s2):
        if self.el_model.test_with_check(s1,s2) == 0:
            return 0
        else:
            return self.el_model.test_with_check(s1,s2)[0][0]

    def check_ner(self):
        flag = 0 #0为不合格 1为单症状 2为多症状 3为单药方
        if ('B-MED' in self.ner_result and 'E-MED' not in self.ner_result) or ('B-SYM' in self.ner_result and 'E-SYM' not in self.ner_result):
            flag = 0
            return flag
  
        if 'B-MED' in self.ner_result and 'B-SYM' not in self.ner_result:
            if self.ner_result.count('B-MED') != 1:
                flag = 0
                return flag
            else:
                flag = 3
                return flag
        elif 'B-MED' in self.ner_result and 'B-SYM' in self.ner_result:
            flag = 0
            return flag
        elif 'B-MED' not in self.ner_result and 'B-SYM' not in self.ner_result:
            flag = 0
            return flag
        elif 'B-MED' not in self.ner_result and 'B-SYM' in self.ner_result:
            if self.ner_result.count('B-SYM') == 1:
                flag = 1
                return flag
            elif self.ner_result.count('B-SYM') > 1:
                flag = 2
                return flag
            else:
                flag = 0
                return flag
        else:
            flag = 0
            return flag

    def extract_ner_data(self):
        flag = self.check_ner()
        if flag == 1:
            self.ner_entity = ''
            self.intent_label_str = list(self.ques)
            begin_index = self.ner_result.index('B-SYM')
            end_index = self.ner_result.index('E-SYM')
            del self.intent_label_str[begin_index+1:end_index+1]
            self.intent_label_str[begin_index]='SYM'
            self.intent_label_str = ''.join(self.intent_label_str)
            last_flag_index = -1
            for cha in self.ner_result:
                if cha != 'O':
                    self.ner_entity += self.ques[self.ner_result.index(cha,last_flag_index + 1)]
                    last_flag_index = self.ner_result.index(cha,last_flag_index + 1)

            return 1
        elif flag == 2:
            self.ner_entity = []
            self.intent_label_str = list(self.ques)
            num = self.ner_result.count('B-SYM')
            temp_count = []
            last_begin_index = self.ner_result.index('B-SYM')
            last_end_index = self.ner_result.index('E-SYM')
            temp_count.append([last_begin_index,last_end_index])
            for count in range(0,num-1):
                temp_count.append([self.ner_result.index('B-SYM',last_begin_index+1),self.ner_result.index('E-SYM',last_end_index+1)])
                last_begin_index = self.ner_result.index('B-SYM',last_begin_index+1)
                last_end_index = self.ner_result.index('E-SYM',last_end_index+1)
                    
            for temp in temp_count:
                self.intent_label_str[temp[0]]='SYM'
                self.intent_label_str[temp[0]+1:temp[1]+1]=' '*(temp[1]-temp[0])
                self.ner_entity.append(self.ques[temp[0]:temp[1]+1])

            for temp in temp_count:
                if ' ' in self.intent_label_str:
                    self.intent_label_str.remove(' ')

            self.intent_label_str = ''.join(self.intent_label_str)
      
            return 1
        elif flag == 3:
            self.ner_entity = ''
            self.intent_label_str = list(self.ques)
            begin_index = self.ner_result.index('B-MED')
            end_index = self.ner_result.index('E-MED')
            del self.intent_label_str[begin_index+1:end_index+1]
            self.intent_label_str[begin_index]='MED'
            self.intent_label_str = ''.join(self.intent_label_str)
            
            last_flag_index = -1
            for cha in self.ner_result:
                if cha != 'O':
                    self.ner_entity += self.ques[self.ner_result.index(cha,last_flag_index + 1)]
                    last_flag_index = self.ner_result.index(cha,last_flag_index + 1)

            return 2
        else:
            return -1
    
    #实体链接
    def kg_el(self,s1):
        #同类型实体 遍历寻找相似度最大且相似度>0.6
        now_max_entity = ''
        score = 0
        for sym in self.qa_sym:
            if sym + '了' == s1:
                return sym,1
            temp = self.el(s1,sym)
            if temp > score and temp > 0.6:
                now_max_entity = sym
                score = temp
            else:
                pass
        if score == 0 and sym in s1:
            return sym,1
        return now_max_entity,score
        
    def kg_connect(self):
        #self.search_dict {'已链接': [('伤寒了', '伤寒')], '意图': 'sym_med', '待链接': ['伤寒了'], '无法链接': []}
        #{'已链接': ['紫丸'], '意图': 'med_use', '待链接': [], '无法链接': []}
        temp_dict = {}
        link_1 = self.search_dict['已链接']
        if self.search_dict['意图'] == 'sym_med':
            for entity in link_1:
                temp_dict[entity] = list(self.kg.Q_sym_med(entity[1])['m.name'])
                
        elif self.search_dict['意图'] == 'med_sym':
            temp_dict[link_1[0]] = list(self.kg.Q_med_sym(link_1[0][1])['m.name'])
            
        elif self.search_dict['意图'] == 'med_con':
            temp_dict[link_1[0]] = [list(self.kg.Q_med_con(link_1[0][1])['m.name']),list(self.kg.Q_med_con(link_1[0][1])['r.分量'])]
            
        elif self.search_dict['意图'] == 'med_use':
            temp_dict[link_1[0]] = list(self.kg.Q_med_use(link_1[0][1])['m.name'])
            
        self.search_dict['kg连接'] = temp_dict
    
    
    def start(self):
        #self.ques = input()
        #if self.ques == 'quit':
        #    return -2
        self.ques = self.ques.strip()
        print('正在检索中···')
        if '、' in self.ques:
            self.ques = self.ques.replace('、','，')
        if ',' in self.ques:
            self.ques = self.ques.replace(',','，')
        if ' ' in self.ques:
            self.ques = self.ques.replace(' ','，')
        error_list = ['还','还有点','有点','还有','一点','还有一点']
        for temp_error in error_list:
            if temp_error in self.ques:
                self.ques = self.ques.replace(temp_error,'，')
                break
        self.ner()
        self.ner_result = self.ner_pred_results[0]
        check_flag = self.extract_ner_data()
        if check_flag == -1:
            print('问句实体数量有误，请调整说法后重新输入。')
            return -1
        self.ir()
        if check_flag == 1 and self.ir_pred_results != 'sym_med':
            print('问句结构有误，请调整实体与意图重新输入。')
            return -1
        elif check_flag == 2 and self.ir_pred_results == 'sym_med':
            print('问句结构有误，请调整实体与意图重新输入。')
            return -1
        
        if self.ir_pred_results == 'unknown':
            print("请调整说法后重新输入。")
            return -1

        self.search_dict = {}
        self.search_dict['已链接'] = []
        self.search_dict['意图'] = self.ir_pred_results
        self.search_dict['待链接'] = []
        self.search_dict['无法链接'] = []
        #单症状 or 多症状 首先判断是否与知识图谱匹配 不匹配的话算相似度
        if check_flag == 1:
          #多症状
            if type(self.ner_entity) == list:
                for sub_sym in self.ner_entity:
                    if sub_sym in self.qa_sym:
                        self.search_dict['已链接'].append((sub_sym,sub_sym))
                    else:
                        self.search_dict['待链接'].append(sub_sym)
                        '''temp_jieba = list(jieba.cut(sub_sym))
                        for sub_sub_sym in temp_jieba:
                            if sub_sub_sym in self.qa_sym:
                                self.serach_dict['已链接'].append((sub_sub_sym,sub_sub_sym))
                            else:
                                self.search_dict['待链接'].append(sub_sub_sym)'''
    
                        
          #单症状
            elif type(self.ner_entity) == str:
                if self.ner_entity in self.qa_sym:
                    self.search_dict['已链接'].append((self.ner_entity,self.ner_entity))
                else:
                    self.search_dict['待链接'].append(self.ner_entity)
                    '''temp_jieba = list(jieba.cut(self.ner_entity))
                    for sub_sub_sym in temp_jieba:
                        if sub_sub_sym in self.qa_sym:
                            self.serach_dict['已链接'].append((sub_sub_sym,sub_sub_sym))
                        else:
                            self.search_dict['待链接'].append(sub_sub_sym)'''
                    
            for s1 in self.search_dict['待链接']:
                now_max_entity,score = self.kg_el(s1)
                if score > 0:
                    self.search_dict['已链接'].append((s1,now_max_entity))
                else:
                    temp_jieba = list(jieba.cut(s1))
                    for sub_sub_sym in temp_jieba:
                        now_max_entity,score = self.kg_el(sub_sub_sym)
                        if score > 0:
                            self.search_dict['已链接'].append((sub_sub_sym,now_max_entity))
                        else:
                            self.search_dict['无法链接'].append(sub_sub_sym)
                
        #单药方
        elif check_flag == 2:
            if self.ner_entity in self.qa_med:
                self.search_dict['已链接'].append((self.ner_entity,self.ner_entity))
            else:
                if self.ner_entity + '方' in self.qa_med:
                    self.search_dict['已链接'].append((self.ner_entity,self.ner_entity+'方'))
                else:
                    print('您输入的实体名称可能有误，请检查后重新输入。')
                    return -1
        #错误
        elif check_flag == -1:
            print('问句结构有误，请调整说法后重新输入（注意：无法对症状和药方进行同时发问）')
            return -1
        
        
    def answer(self):
        print_str = ''
        self.kg_connect()
        if self.search_dict['意图'] == 'sym_med':
            if len(self.search_dict['kg连接']) == 1:
                sym_entity = list(self.search_dict['kg连接'].keys())[0][0]
                print_str += sym_entity
                print_str += '可以使用'
                lst = self.search_dict['kg连接'][list(self.search_dict['kg连接'].keys())[0]]
                for i in range(0,len(lst)):
                    print_str += lst[i]
                    if i < len(lst)-1 :
                        print_str += '、'
                print_str += '来进行治疗。'
                print(print_str)
                print_str = ''
                
                if self.search_dict['无法链接'] != []:
                    for false in range(0,len(self.search_dict['无法链接'])):
                        print_str += self.search_dict['无法链接'][false]
                        if false <  len(self.search_dict['无法链接'])-1:
                            print_str += '、'
                    print_str += '在知识图谱中无对应结果。'
                    print(print_str)
            elif len(self.search_dict['kg连接']) > 1:
                sym_entity_lst = []
                med_lst = []
                for key in self.search_dict['kg连接'].keys():
                    sym_entity_lst.append(key[0])
                    med_lst.append(self.search_dict['kg连接'][key])
                #多个都有重的的话 返回重的 没有的话就每个都列出来    
                inter = med_lst[0]
                for temp_med_lst in med_lst[1:]:
                    inter = set(inter).intersection(temp_med_lst)
                    
                if len(list(inter)) >= 1:
                    for sym_index in range(0,len(sym_entity_lst)):
                        print_str += sym_entity_lst[sym_index]
                        if sym_index < len(sym_entity_lst)-1:
                            print_str += '、'
                    print_str += '可以使用'
                    
                    for med_index in range(0,len(list(inter))):
                        print_str += list(inter)[med_index]
                        if med_index < len(list(inter))-1:
                            print_str += '、'
                    print_str += '来进行治疗。'
                    print(print_str)
                    print_str = ''
                else:#没有重的 每个单独列出来
                    for sym_index in range(0,len(sym_entity_lst)):
                        print_str += sym_entity_lst[sym_index]
                        print_str += '可以使用'
                        for min_med_index in range(0,len(med_lst[sym_index])):
                            print_str += med_lst[sym_index][min_med_index]
                            if min_med_index < len(med_lst[sym_index])-1:
                                print_str += '、'
                        print_str += '来进行治疗。'
                        print(print_str)
                        print_str = ''
                    
            if self.search_dict['无法链接'] != []:
                for false in range(0,len(self.search_dict['无法链接'])):
                    print_str += self.search_dict['无法链接'][false]
                    if false <  len(self.search_dict['无法链接'])-1:
                        print_str += '、'
                print_str += '在知识图谱中无对应结果。'
                print(print_str)
                print_str = ''
                    
        elif self.search_dict['意图'] == 'med_sym':
            if len(self.search_dict['kg连接']) == 1:
                med_entity = list(self.search_dict['kg连接'].keys())[0][0]
                print_str += med_entity
                print_str += '可以用来治疗：'
                lst = self.search_dict['kg连接'][list(self.search_dict['kg连接'].keys())[0]]
                for i in range(0,len(lst)):
                    print_str += lst[i]
                    if i < len(lst)-1 :
                        print_str += '、'
                print_str += '。'
                print(print_str)
                print_str = ''
                
            if self.search_dict['无法链接'] != []:
                for false in range(0,len(self.search_dict['无法链接'])):
                    print_str += self.search_dict['无法链接'][false]
                    if false <  len(self.search_dict['无法链接'])-1:
                        print_str += '、'
                print_str += '在知识图谱中无对应结果。'
                print(print_str)
                print_str = ''
            
            
        elif self.search_dict['意图'] == 'med_con':
            if len(self.search_dict['kg连接']) == 1:
                med_entity = list(self.search_dict['kg连接'].keys())[0][0]
                print_str += med_entity
                print_str += '的成分和具体剂量为：'
                lst = self.search_dict['kg连接'][list(self.search_dict['kg连接'].keys())[0]]
                for i in range(0,len(lst[0])):
                    print_str += lst[0][i]
                    print_str += '-'
                    print_str += lst[1][i]
                    if i < len(lst[0])-1 :
                        print_str += '，'
                print_str += '。'
                print(print_str)
                print_str = ''
                
            if self.search_dict['无法链接'] != []:
                for false in range(0,len(self.search_dict['无法链接'])):
                    print_str += self.search_dict['无法链接'][false]
                    if false <  len(self.search_dict['无法链接'])-1:
                        print_str += '、'
                print_str += '在知识图谱中无对应结果。'
                print(print_str)
                print_str = ''
                
        elif self.search_dict['意图'] == 'med_use':
            if len(self.search_dict['kg连接']) == 1:
                med_entity = list(self.search_dict['kg连接'].keys())[0][0]
                print_str += med_entity
                print_str += '的具体使用方法是：'
                lst = self.search_dict['kg连接'][list(self.search_dict['kg连接'].keys())[0]]
                for i in range(0,len(lst)):
                    print_str += lst[i]
                    if i < len(lst)-1 :
                        print_str += '，'
                print_str += '。'
                print(print_str)
                print_str = ''
                
            if self.search_dict['无法链接'] != []:
                for false in range(0,len(self.search_dict['无法链接'])):
                    print_str += self.search_dict['无法链接'][false]
                    if false <  len(self.search_dict['无法链接'])-1:
                        print_str += '、'
                print_str += '在知识图谱中无对应结果。'
                print(print_str)
                print_str = ''
                
                
    def final_start(self):
        print('欢迎进入中医儿科症状治法知识库问答系统，请注意以下要求：')
        print('1.症状实体可用中文逗号连接进行一次性多个输入。')
        print('2.一次只可询问一个药方的内容、剂量或主治。')
        print('3.无法同时询问症状和药方。')
        
        while True:
            print('-------------------------------------------------')
            print('请输入您的问题（退出请输入quit）：')
            self.ques = input()
            if self.ques == 'quit':
                break
            elif self.start() == -1:
                break
            else:
                self.answer()

In [7]:
test = TCM_QA()

正在进行系统初始化···


In [15]:
test.final_start()

欢迎进入中医儿科症状治法知识库问答系统，请注意以下要求：
1.症状实体可用中文逗号连接进行一次性多个输入。
2.一次只可询问一个药方的内容、剂量或主治。
3.无法同时询问症状和药方。
-------------------------------------------------
请输入您的问题（退出请输入quit）：
孩子感冒发烧怎么办
正在检索中···
感冒、发烧可以使用桂枝汤方来进行治疗。
-------------------------------------------------
请输入您的问题（退出请输入quit）：
quit


In [11]:
test.search_dict

{'已链接': [('感冒', '鼻鸣'), ('发烧', '鼻鸣')],
 '意图': 'sym_med',
 '待链接': ['感冒', '发烧'],
 '无法链接': [],
 'kg连接': {('感冒', '鼻鸣'): ['桂枝汤方'], ('发烧', '鼻鸣'): ['桂枝汤方']}}

In [ ]:
紫丸一般可以用来治疗什么？

In [ ]:
二物驴毛散配好药以后应该怎么做？

In [ ]:
#孩子伤寒了怎么治效果会比较好？
#我呕吐，鼻塞，还尿频吃什么药比较好啊
#伤寒，怀孕，口干呕吐吃什么药能好啊
#我呕吐鼻塞，还尿频吃什么药比较好啊
#伤寒了怎么治效果会比较好？
#伤寒了用紫丸治会好点吗
#紫丸应该怎么做
#少小客忤如何改善？
#二物驴毛散和紫丸方都能治什么症状呢
#二物驴毛散，紫丸方都能治什么症状呢
#伤寒，孕吐，可以用紫丸治吗


In [ ]:
test.ner_result

In [ ]:
test.search_dict

In [ ]:
test.kg_el('口干')

In [ ]:
test.kg.Q_med_con(test.search_dict['已链接'][0][1])['m.name']

In [ ]:
test.kg.Q_med_con(test.search_dict['已链接'][0][1])['r.分量']

In [ ]:
lst = test.search_dict['kg连接'][list(test.search_dict['kg连接'].keys())[0]]

In [ ]:
print_str = ''

In [ ]:
for i in range(0,len(lst)):
    print_str += lst[0][i]
    print_str += '-'
    print_str += lst[1][i]
    if i < len(lst)-1 :
        print_str += '，'
    print_str += '。'
    print(print_str)
    print_str = ''